In [1]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
print(selenium.__version__)

4.8.2


In [4]:
chrome_options = Options()

chrome_options.add_experimental_option("detach", True)
chrome_options.add_experimental_option("excludeSwitches",["enable-logging"])

driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options=chrome_options)

In [5]:
# 무신사 링크
url = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1'
driver.get(url)
driver.maximize_window() # 브라우저의 크기 전체화면으로 확대

In [23]:
columns = ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers']         # 코디맵 카테고리, 제목, 일자, 조회수, 댓글수 크롤링
values = []                                             # 코디맵 정보를 담을 빈 리스트 생성
soup = BeautifulSoup(driver.page_source, 'lxml')        # 현재 브라우저의 page의 html 정보를 lxml을 이용하여 가져오기
data_rows = soup.find_all('li', attrs={'class':'style-list-item'}) # 해당 조건에 맞는 정보들을 list로 반환

for i, row in enumerate(data_rows):                     # data_rows 리스트에 있는 값들과 인덱스를 차례대로 가져온다

    print('{0}번째 코디맵 크롤링'.format(i+1))
    blank = []                                          # 코디맵 정보를 담을 빈 리스트 생성

    codimap_category = row.find('span', attrs={'class':'style-list-information__text'})       # 코디맵 카테고리 가져오기
    if codimap_category:                                                                    # 해당 조건에 맞는 정보가 하나라도 있을 경우:
        codimap_category = codimap_category.get_text().strip()                                          # html에서 텍스트만 가져온 뒤 공백 제거
        blank.append(codimap_category)
    else:                                                                       # 해당 조건에 맞는 정보가 하나도 없을 경우:
        blank.append('Something is wrong')
        print('{}번째 카테고리 가져올때 문제발생'.format(i+1))
        continue


    codimap_title = row.find('strong', attrs={'class':'style-list-information__title'})      # 제목 가져오기
    if  codimap_title:
        codimap_title =  codimap_title.get_text().strip()
        blank.append( codimap_title)
    else:
        blank.append('Something is wrong')
        print('{}번째 제목 가져올때 문제발생'.format(i+1))
        continue

    all = row.find_all('span', attrs={'class':'post-information__text'})  

    codimap_date = all[0]       # 날짜 가져오기
    if codimap_date:
        codimap_date = codimap_date.get_text().strip()    # class가 Rating__star_area--dFzsb인 span 태그 안에 있는 span 태그에서 텍스트 가져오기
        blank.append(codimap_date)
    else:
        blank.append('Something is wrong')
        print('{}번째 날짜 가져올때 문제발생'.format(i+1))
        continue

    # 조회수 가져오기
    views_element = all[1]
    if views_element:
        views_count = views_element.get_text(strip=True).split()[1]  # "조회 17,219"에서 숫자 부분만 추출
        blank.append(views_count)
    else:
        blank.append('Views count not found')
        print('{}번째 조회수 가져올 때 문제 발생'.format(i+1))
        continue
    


    # 댓글 수 가져오기
    comments_element = all[2]
    print(comments_element)
    if comments_element:
        comment_count = comments_element.get_text(strip=True).split()[1]
        blank.append(comment_count)
    else:
        blank.append('Comments count not found')
        print('{}번째 댓글 수 가져올 때 문제 발생'.format(i+1))
        continue


    values.append(blank)
    print('---------------------------------------------------')

df = pd.DataFrame(values, columns=columns)
df


1번째 코디맵 크롤링
<span class="post-information__text">댓글 50</span>
---------------------------------------------------
2번째 코디맵 크롤링
<span class="post-information__text">댓글 47</span>
---------------------------------------------------
3번째 코디맵 크롤링
<span class="post-information__text">댓글 43</span>
---------------------------------------------------
4번째 코디맵 크롤링
<span class="post-information__text">댓글 32</span>
---------------------------------------------------
5번째 코디맵 크롤링
<span class="post-information__text">댓글 25</span>
---------------------------------------------------
6번째 코디맵 크롤링
<span class="post-information__text">댓글 24</span>
---------------------------------------------------
7번째 코디맵 크롤링
<span class="post-information__text">댓글 19</span>
---------------------------------------------------
8번째 코디맵 크롤링
<span class="post-information__text">댓글 18</span>
---------------------------------------------------
9번째 코디맵 크롤링
<span class="post-information__text">댓글 17</span>
--------------------------

,codimap_category,codimap_title,codimap_date,views,comment_numbers
0,캐주얼,톤온톤 코디,21.07.02,"17,219",50
1,캐주얼,블랙 매니아,21.02.23,"36,280",47
2,캐주얼,오늘의 코디,22.03.22,"13,368",43
3,댄디,추천 댄디 룩,21.09.15,"21,569",32
4,캐주얼,릴랙스 룩,21.02.23,"24,007",25
5,스트릿,힙하다 힙해,21.02.03,"36,056",24
6,댄디,데이트 코디,21.12.24,"10,949",19
7,캐주얼,심플한 코디,21.07.30,"41,334",18
8,댄디,훈훈 남친 룩,21.03.24,"19,978",17
9,댄디,서머 데이트 룩,21.07.26,"8,350",17


In [27]:
# DataFrame을 CSV 파일로 저장
df.to_csv('/Users/daniel/Downloads/codimap_data.csv', index=False, encoding='euc-kr')
# DataFrame을 CSV 파일로 저장 (euc-kr 인코딩 사용)
#df.to_csv('codimap_data.csv', index=False, encoding='euc-kr')
